In [0]:
%run "/Workspace/Users/avinashpayak@gmail.com/Common"

In [0]:
env = "dev"


### Reading from bronze raw_Roads

In [0]:
def read_BronzeRoadsTable(environment):
    print('Reading the Bronze Table raw_roads Data : ',end='')
    df_bronzeRoads = (spark.readStream
                    .table(f"`{environment}-catalog`.`bronze`.raw_roads")
                    )
    print(f'Reading {environment}-catalog.bronze.raw_roads Success!')
    print("**********************************")
    return df_bronzeRoads

In [0]:
df_roads = read_BronzeRoadsTable(env)

Reading the Bronze Table raw_roads Data : Reading dev-catalog.bronze.raw_roads Success!
**********************************



### Creating road_category_name column

In [0]:
def road_Category(df):
    print('Creating Road Category Name Column: ', end='')
    from pyspark.sql.functions import when,col

    df_road_Cat = df.withColumn("Road_Category_Name",
                  when(col('Road_Category') == 'TA', 'Class A Trunk Road')
                  .when(col('Road_Category') == 'TM', 'Class A Trunk Motor')
                   .when(col('Road_Category') == 'PA','Class A Principal road')
                    .when(col('Road_Category') == 'PM','Class A Principal Motorway')
                    .when(col('Road_Category') == 'M','Class B road')
                    .otherwise('NA')
                  
                  )
    print('Success!! ')
    print('***********************')
    return df_road_Cat


### Creating road_type column

In [0]:
def road_Type(df):
    print('Creating Road Type Name Column: ', end='')
    from pyspark.sql.functions import when,col

    df_road_Type = df.withColumn("Road_Type",
                  when(col('Road_Category_Name').like('%Class A%'),'Major')
                  .when(col('Road_Category_Name').like('%Class B%'),'Minor')
                    .otherwise('NA')
                  
                  )
    print('Success!! ')
    print('***********************')
    return df_road_Type


### Creating Writing data to silver_roads in Silver schema

In [0]:
def write_Roads_SilverTable(StreamingDF,environment):
    print('Writing the silver_roads Data : ',end='') 

    write_StreamSilver_R = (StreamingDF.writeStream
                .format('delta')
                .option('checkpointLocation',checkpoint+ "/SilverRoadsLoad/Checkpt/")
                .outputMode('append')
                .queryName("SilverRoadsWriteStream")
                .trigger(availableNow=True)
                .toTable(f"`{environment}-catalog`.`silver`.`silver_roads`"))
    
    write_StreamSilver_R.awaitTermination()
    print(f'Writing `{environment}-catalog`.`silver`.`silver_roads` Success!')


### Calling functions

In [0]:
df_noDups = remove_Dups(df_roads)

AllColumns = df_noDups.schema.names
df_clean = handle_NULLs(df_noDups,AllColumns)

## Creating Road_Category_name 
df_roadCat = road_Category(df_clean)

## Creating Road_Type column
df_type = road_Type(df_roadCat)

## Writing data to silver_roads table

write_Roads_SilverTable(df_type,env)

Removing Duplicate values: Success!
Replacing NULLs of Strings DataType with "Unknown": Success!
Replacing NULLs of Numeric DataType with "0":  Success!
Creating Road Category Name Column: Success!! 
***********************
Creating Road Type Name Column: Success!! 
***********************
Writing the silver_roads Data : Writing `dev-catalog`.`silver`.`silver_roads` Success!
